<a href="https://colab.research.google.com/github/luisosmx/API_connections/blob/main/CoreLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import json


class PropertySearch:
    def __init__(self, base_url, client_credentials_url, client_id, client_secret):
        self.base_url = base_url
        self.client_credentials_url = client_credentials_url
        self.client_id = client_id
        self.client_secret = client_secret
        self.token = None

    def authenticate(self):
        auth_data = "grant_type=client_credentials"

        auth_response = requests.post(
            self.client_credentials_url + "?" + auth_data,
            auth=(self.client_id, self.client_secret),
            # data=auth_data
        )

        if auth_response.status_code == 200:
            auth_response_data = auth_response.json()
            self.token = auth_response_data.get("access_token")
            print(f"Authentication successful.")
        else:
            print(f"Authentication failed. Status Code: {auth_response.status_code}")

    def search_property(self, street_address, city, state, zip_code):
        if not self.token:
            print("Please authenticate first.")
            return None

        self.headers = {
                "Authorization": f"Bearer {self.token}",
                "Content-Type": "application/json"
            }

        params = {
            "streetAddress": street_address,
            "city": city,
            "state": state,
            "zipCode": zip_code
        }

        self.response = requests.get(self.base_url, headers=self.headers, params=params)

        if self.response.status_code == 200:
            return self.response.json()
        else:
            print(f"Error: {self.response.status_code}")
            return None

    def property_detail(self, response):

        self.clip_value = response['items'][0]['clip']
        url = f'https://property.corelogicapi.com/v2/properties/{self.clip_value}/property-detail'
        self.resource_detail = requests.get(url, headers= self.headers)

        if self.resource_detail.status_code == 200:
              return self.resource_detail.json()
        else:
            print(f"Error: {self.resource_detail.status_code}")
            return None

    def ownership_detail(self):
        url = f'https://property.corelogicapi.com/v2/properties/{self.clip_value}/ownership'
        self.source_ownership = requests.get(url, headers= self.headers)
        if self.source_ownership.status_code == 200:
              return self.source_ownership.json()
        else:
            print(f"Error: {self.source_ownership.status_code}")
            return None

    def liens_detail(self):
        url = f'https://property.corelogicapi.com/v2/properties/{self.clip_value}/liens'
        self.source_liens = requests.get(url, headers= self.headers)
        if self.source_liens.status_code == 200:
              return self.source_liens.json()
        else:
            print(f"Error: {self.source_liens.status_code}")
            return None


    def extract_relevant_data(self, response, resource_detail):
        try:
            building_data = {
                'clip': response['items'][0]['clip'],
                'yearBuilt': resource_detail['buildings']['data']['buildings'][0]['constructionDetails']['yearBuilt'],
                'streetAddress': response['items'][0]['propertyAddress']['streetAddress'],
                'city': response['items'][0]['propertyAddress']['city'],
                'state': response['items'][0]['propertyAddress']['state'],
                'zipCode': response['items'][0]['propertyAddress']['zipCode'],
                'propertyMatchScore': response['items'][0]['addressMatchInformation']['propertyMatchScore'],
                'Tax_Year': resource_detail['taxAssessment']['items'][0]['assessedValue']['taxAssessedYear'],
                'Total_Tax_Amount': resource_detail['taxAssessment']['items'][0]['taxAmount']['totalTaxAmount'],
                'Land_Value': resource_detail['taxAssessment']['items'][0]['assessedValue']['calculatedLandValue'],
                'Total_Value': resource_detail['taxAssessment']['items'][0]['assessedValue']['calculatedTotalValue'],
                'Improvement_Value': resource_detail['taxAssessment']['items'][0]['assessedValue']['calculatedImprovementValue']
            }

            building_df = pd.DataFrame([building_data])

            return building_df

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None

client_id = ''
client_secret = ''
property_search_url = "https://property.corelogicapi.com/v2/properties/search"
auth_url = "https://api-prod.corelogic.com/oauth/token"

try:
    property_search = PropertySearch(property_search_url, auth_url, client_id, client_secret)

    # Authenticate to obtain the access token
    property_search.authenticate()

    # Now you can use the search_property method
    result = property_search.search_property("31899 Highway 41", "Coarsegold", "CA", "93614")

    if result:
        print("Property Search Result:")
        #print(json.dumps(result, indent=4))
        print(result)

        resource_url = property_search.property_detail(result)
        print('Property Detail:')
        #print(json.dumps(resource_url, indent=4))
        print(resource_url)

        resource_ownership = property_search.ownership_detail()
        print('Ownership Detail:')
        #print(json.dumps(resource_ownership, indent=4))
        print(resource_ownership)

        resource_liens = property_search.liens_detail()
        print('Liens Detail:')
        print(resource_liens)



        result_df = property_search.extract_relevant_data(result, resource_url)
        print(result_df)
except Exception as e:
    print(f"An error occurred: {str(e)}")